### Scrape MNREGA Social Audit Data

In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    StaleElementReferenceException,
    ElementClickInterceptedException,
    NoSuchElementException
)
from datetime import datetime
from pathlib import Path
import gzip

In [2]:
class FormHandler:
    def __init__(self, url: str):
        self.options = Options()
        self.options.add_argument('-headless')
        self.driver = webdriver.Firefox(options=self.options)
        self.driver.get(url)
        self.wait = WebDriverWait(self.driver, 20)
        
        self.form_elements = {
            'state': 'ctl00_ContentPlaceHolder1_ddlstate',
            'district': 'ctl00_ContentPlaceHolder1_ddldistrict',
            'block': 'ctl00_ContentPlaceHolder1_ddlBlock',
            'panchayat': 'ctl00_ContentPlaceHolder1_ddlPanchayat',
            'year': 'ctl00_ContentPlaceHolder1_ddlAuditYear',
            'date': 'ctl00_ContentPlaceHolder1_ddlGSDate',
            'option': 'ctl00_ContentPlaceHolder1_ddlselect'
        }
        
        self.html_dir = Path('../data/html')
        self.html_dir.mkdir(exist_ok=True)
        
        self.results = []
        self.total_processed = 0
        self.save_interval = 10  # Save intermediate CSV every 10 rows
        self.resume_file = "../data/intermediate_results.csv"
        self.processed_set = self._load_processed_set()
    
    def _load_processed_set(self) -> set:
        """Load processed combination keys from the intermediate CSV (if it exists),
        and update self.results and self.total_processed.
        """
        processed = set()
        if os.path.exists(self.resume_file):
            try:
                df_existing = pd.read_csv(self.resume_file)
                self.results = df_existing.to_dict('records')
                self.total_processed = len(self.results)
                for _, row in df_existing.iterrows():
                    key = (
                        str(row['state_val']).strip(),
                        str(row['district_val']).strip(),
                        str(row['block_val']).strip(),
                        str(row['panchayat_val']).strip(),
                        str(row['year_val']).strip(),
                        str(row['date_val']).strip(),
                        str(row['option_val']).strip()
                    )
                    processed.add(key)
                print(f"Resuming from {len(processed)} processed combinations.")
            except Exception as e:
                print("Error loading intermediate results:", e)
        return processed
    
    def _wait_for_enabled(self, element_id: str) -> Select:
        """Wait for a dropdown element to be clickable and return a Select object."""
        element = self.wait.until(EC.element_to_be_clickable((By.ID, element_id)))
        return Select(element)
    
    def _get_options_dict(self, element_id: str, exclude_default=True) -> dict:
        """
        Return a dictionary mapping option values to their visible text (labels)
        for the given dropdown element.
        If exclude_default is True, options with value "0" are omitted.
        Retries a few times if the element becomes stale or times out.
        """
        max_retries = 3
        attempts = 0
        while attempts < max_retries:
            try:
                select = self._wait_for_enabled(element_id)
                if exclude_default:
                    options = {opt.get_attribute('value'): opt.text.strip()
                               for opt in select.options if opt.get_attribute('value') != '0'}
                else:
                    options = {opt.get_attribute('value'): opt.text.strip()
                               for opt in select.options}
                return options
            except (StaleElementReferenceException, TimeoutException) as e:
                attempts += 1
                print(f"{e.__class__.__name__} in _get_options_dict for {element_id}. Retrying {attempts} of {max_retries} ...")
                time.sleep(2)
        print(f"Error: Could not get options for {element_id} after {max_retries} attempts. Skipping this combination.")
        return {}
    
    def _select_option(self, element_id: str, value: str, max_retries=3):
        """
        Select an option in the dropdown by its value and wait briefly for the page to update.
        If the element is obscured, scroll it into view and retry.
        """
        for attempt in range(max_retries):
            try:
                select = self._wait_for_enabled(element_id)
                select.select_by_value(value)
                time.sleep(2)
                return
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException on {element_id} with value {value}. "
                      f"Attempt {attempt+1} of {max_retries}. Scrolling into view.")
                element = self.driver.find_element(By.ID, element_id)
                self.driver.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(2)
            except Exception as e:
                if attempt == max_retries - 1:
                    raise
                print(f"Error selecting option {value} for element {element_id}: {e}")
                time.sleep(2)
    
    def _save_webpage_old(self, row: dict) -> str:
        """Save the current page HTML to disk and return the filename (or link)."""
        filename = (
            f"{row['state_val']}_{row['district_val']}_{row['block_val']}_"
            f"{row['panchayat_val']}_{row['year_val']}_{row['date_val']}_{row['option_val']}.html"
        )
        filename = "".join(c if c.isalnum() or c in ['_', '-', '.'] else "_" for c in filename)
        filepath = self.html_dir / filename
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(self.driver.page_source)
        return str(filepath)
    
    def _save_webpage(self, row: dict) -> str:
        """Save the current page HTML to disk as gzip and return the filename."""
        filename = (
            f"{row['state_val']}_{row['district_val']}_{row['block_val']}_"
            f"{row['panchayat_val']}_{row['year_val']}_{row['date_val']}_{row['option_val']}.html.gz"
        )
        # Clean filename
        filename = "".join(c if c.isalnum() or c in ['_', '-', '.'] else "_" for c in filename)
        filepath = self.html_dir / filename

        try:
            # Get page source and encode to bytes
            html_content = self.driver.page_source.encode('utf-8')

            # Write compressed content
            with gzip.open(filepath, 'wb') as f:
                f.write(html_content)

            return str(filepath)
        except Exception as e:
            print(f"Error saving compressed HTML: {e}")
            return ""
        
    def iterate_form(self) -> pd.DataFrame:
        """
        Iterate over all nested dropdown combinations and collect a long-form DataFrame.
        Resumes by skipping combinations already processed.
        """
        state_dict = self._get_options_dict(self.form_elements['state'])
        for state_val, state_label in state_dict.items():
            self._select_option(self.form_elements['state'], state_val)
            district_dict = self._get_options_dict(self.form_elements['district'])
            for district_val, district_label in district_dict.items():
                self._select_option(self.form_elements['district'], district_val)
                block_dict = self._get_options_dict(self.form_elements['block'])
                for block_val, block_label in block_dict.items():
                    self._select_option(self.form_elements['block'], block_val)
                    panchayat_dict = self._get_options_dict(self.form_elements['panchayat'])
                    for panchayat_val, panchayat_label in panchayat_dict.items():
                        self._select_option(self.form_elements['panchayat'], panchayat_val)
                        year_dict = self._get_options_dict(self.form_elements['year'])
                        for year_val, year_label in year_dict.items():
                            self._select_option(self.form_elements['year'], year_val)
                            date_dict = self._get_options_dict(self.form_elements['date'])
                            for date_val, date_label in date_dict.items():
                                try:
                                    self._select_option(self.form_elements['date'], date_val)
                                except NoSuchElementException:
                                    print(f"Cannot locate date option with value: {date_val}. Skipping this date.")
                                    continue
                                # For the final "option" dropdown, include all options (do not exclude default)
                                option_dict = self._get_options_dict(self.form_elements['option'], exclude_default=False)
                                option_val, option_label = None, None
                                # Search for an option with label "all" (case-insensitive)
                                for k, v in option_dict.items():
                                    if v.lower() == "all":
                                        option_val, option_label = k, v
                                        break
                                if option_val is None:
                                    if option_dict:
                                        option_val, option_label = list(option_dict.items())[0]
                                    else:
                                        print("No option found for the 'option' dropdown; skipping combination.")
                                        continue
                                
                                key = (state_val.strip(), district_val.strip(), block_val.strip(),
                                       panchayat_val.strip(), year_val.strip(), date_val.strip(),
                                       option_val.strip())
                                if key in self.processed_set:
                                    print(f"Combination already processed, skipping: {key}")
                                    continue
                                
                                max_attempts = 3
                                attempts = 0
                                while attempts < max_attempts:
                                    try:
                                        self._select_option(self.form_elements['option'], option_val)
                                        html_file = self._save_webpage({
                                            'state_val': state_val,
                                            'district_val': district_val,
                                            'block_val': block_val,
                                            'panchayat_val': panchayat_val,
                                            'year_val': year_val,
                                            'date_val': date_val,
                                            'option_val': option_val
                                        })
                                        row = {
                                            'state_val': state_val,
                                            'state_label': state_label,
                                            'district_val': district_val,
                                            'district_label': district_label,
                                            'block_val': block_val,
                                            'block_label': block_label,
                                            'panchayat_val': panchayat_val,
                                            'panchayat_label': panchayat_label,
                                            'year_val': year_val,
                                            'year_label': year_label,
                                            'date_val': date_val,
                                            'date_label': date_label,
                                            'option_val': option_val,
                                            'option_label': option_label,
                                            'html_file': html_file,
                                            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                        }
                                        self.results.append(row)
                                        self.total_processed += 1
                                        self.processed_set.add(key)
                                        
                                        print(f"[{self.total_processed}] Processed: State: {state_label} ({state_val}), "
                                              f"District: {district_label} ({district_val}), Block: {block_label} ({block_val}), "
                                              f"Panchayat: {panchayat_label} ({panchayat_val}), Year: {year_label} ({year_val}), "
                                              f"Date: {date_label} ({date_val}), Option: {option_label} ({option_val}). "
                                              f"HTML saved to {html_file}")
                                        if self.total_processed % self.save_interval == 0:
                                            pd.DataFrame(self.results).to_csv(self.resume_file, index=False)
                                            print(f"Intermediate results saved with {len(self.results)} rows.")
                                        break  # Success: exit retry loop
                                    except NoSuchElementException as ne:
                                        print(f"NoSuchElementException for combination {key}: Option '{option_val}' not found. Skipping this combination.")
                                        break
                                    except (StaleElementReferenceException, TimeoutException) as e:
                                        attempts += 1
                                        print(f"{e.__class__.__name__} for combination {key}. Retrying {attempts} of {max_attempts} ...")
                                        self.driver.refresh()
                                        time.sleep(3)
                                        try:
                                            self._select_option(self.form_elements['state'], state_val)
                                            self._select_option(self.form_elements['district'], district_val)
                                            self._select_option(self.form_elements['block'], block_val)
                                            self._select_option(self.form_elements['panchayat'], panchayat_val)
                                            self._select_option(self.form_elements['year'], year_val)
                                            self._select_option(self.form_elements['date'], date_val)
                                        except Exception as e2:
                                            print("Error re-selecting after refresh:", e2)
                                    if attempts == max_attempts:
                                        print(f"Maximum attempts reached for combination {key}. Skipping.")
                                # End of retry loop for this combination
        return pd.DataFrame(self.results)
    
    def cleanup(self):
        self.driver.quit()

In [ ]:
url = "https://mnregaweb4.nic.in/netnrega/SocialAuditFindings/SA-GPReport.aspx?page=S&lflag=eng"
handler = FormHandler(url)
try:
    df = handler.iterate_form()
    output_file = f'final_results_{datetime.now().strftime("%Y%m%d_%H%M")}.csv'
    df.to_csv(output_file, index=False)
    print(f"Final results saved to {output_file}")
finally:
    handler.cleanup()

Resuming from 12897 processed combinations.
[14461] Processed: State: ANDHRA PRADESH (02), District: ALLURI SITHARAMA RAJU (0214), Block: Addateegala (0204003), Panchayat: ADDATEEGALA (0204003012), Year: 2023-2024 (2023-2024), Date: 07/07/2024 (7/7/2024 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0214_0204003_0204003012_2023-2024_7_7_2024_12_00_00_AM_0.html.gz
[14462] Processed: State: ANDHRA PRADESH (02), District: ALLURI SITHARAMA RAJU (0214), Block: Addateegala (0204003), Panchayat: ADDATEEGALA (0204003012), Year: 2022-2023 (2022-2023), Date: 15/05/2023 (5/15/2023 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0214_0204003_0204003012_2022-2023_5_15_2023_12_00_00_AM_0.html.gz
[14463] Processed: State: ANDHRA PRADESH (02), District: ALLURI SITHARAMA RAJU (0214), Block: Addateegala (0204003), Panchayat: ADDATEEGALA (0204003012), Year: 2021-2022 (2021-2022), Date: 23/05/2022 (5/23/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0214_02040

ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2017-2018. Attempt 2 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2017-2018. Attempt 3 of 3. Scrolling into view.
[14481] Processed: State: ANDHRA PRADESH (02), District: ALLURI SITHARAMA RAJU (0214), Block: Addateegala (0204003), Panchayat: BODLANKA (0204003014), Year: 2017-2018 (2017-2018), Date: 13/05/2023 (5/13/2023 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0214_0204003_0204003014_2017-2018_5_13_2023_12_00_00_AM_0.html.gz
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlPanchayat with value 0204003006. Attempt 1 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlPanchayat with value 0204003006. Attempt 2 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlPanchayat with value 0204003006. Attempt 3 of 3. Scrolling int

[14497] Processed: State: ANDHRA PRADESH (02), District: ALLURI SITHARAMA RAJU (0214), Block: Addateegala (0204003), Panchayat: DAKODU (0204003003), Year: 2022-2023 (2022-2023), Date: 10/05/2023 (5/10/2023 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0214_0204003_0204003003_2022-2023_5_10_2023_12_00_00_AM_0.html.gz
[14498] Processed: State: ANDHRA PRADESH (02), District: ALLURI SITHARAMA RAJU (0214), Block: Addateegala (0204003), Panchayat: DAKODU (0204003003), Year: 2021-2022 (2021-2022), Date: 19/05/2022 (5/19/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0214_0204003_0204003003_2021-2022_5_19_2022_12_00_00_AM_0.html.gz
[14499] Processed: State: ANDHRA PRADESH (02), District: ALLURI SITHARAMA RAJU (0214), Block: Addateegala (0204003), Panchayat: DAKODU (0204003003), Year: 2020-2021 (2020-2021), Date: 25/05/2022 (5/25/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0214_0204003_0204003003_2020-2021_5_25_2022_12_00_00_AM_0.html.gz
[

ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2022-2023. Attempt 2 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2022-2023. Attempt 3 of 3. Scrolling into view.
[14519] Processed: State: ANDHRA PRADESH (02), District: ALLURI SITHARAMA RAJU (0214), Block: Addateegala (0204003), Panchayat: DUCHERTHI (0204003002), Year: 2022-2023 (2022-2023), Date: 30/08/2018 (8/30/2018 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0214_0204003_0204003002_2022-2023_8_30_2018_12_00_00_AM_0.html.gz
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2021-2022. Attempt 1 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2021-2022. Attempt 2 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2021-2022. Attempt 3 of 3. Scrolling into 

ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2022-2023. Attempt 2 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2022-2023. Attempt 3 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2021-2022. Attempt 1 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2021-2022. Attempt 2 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2021-2022. Attempt 3 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2020-2021. Attempt 1 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_ddlAuditYear with value 2020-2021. Attempt 2 of 3. Scrolling into view.
ElementClickInterceptedException on ctl00_ContentPlaceHolder1_

...

[21100] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: KRISHNAPURAM (0203041016), Year: 2021-2022 (2021-2022), Date: 14/09/2022 (9/14/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041016_2021-2022_9_14_2022_12_00_00_AM_0.html.gz
Intermediate results saved with 21100 rows.
[21101] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: KRISHNAPURAM (0203041016), Year: 2020-2021 (2020-2021), Date: 13/09/2022 (9/13/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041016_2020-2021_9_13_2022_12_00_00_AM_0.html.gz
[21102] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: KRISHNAPURAM (0203041016), Year: 2019-2020 (2019-2020), Date: 12/09/2022 (9/12/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041016_2019-2020_9

TimeoutException in _get_options_dict for ctl00_ContentPlaceHolder1_ddlselect. Retrying 1 of 3 ...
TimeoutException in _get_options_dict for ctl00_ContentPlaceHolder1_ddlselect. Retrying 2 of 3 ...
TimeoutException in _get_options_dict for ctl00_ContentPlaceHolder1_ddlselect. Retrying 3 of 3 ...
Error: Could not get options for ctl00_ContentPlaceHolder1_ddlselect after 3 attempts. Skipping this combination.
No option found for the 'option' dropdown; skipping combination.
[21120] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: Padmanabharaju peta (0203041020), Year: 2020-2021 (2020-2021), Date: 06/09/2022 (9/6/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041020_2020-2021_9_6_2022_12_00_00_AM_0.html.gz
Intermediate results saved with 21120 rows.
[21121] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: Padmanabharaju peta (0203041

[21143] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: Purusotha puram (0203041014), Year: 2018-2019 (2018-2019), Date: 04/02/2020 (2/4/2020 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041014_2018-2019_2_4_2020_12_00_00_AM_0.html.gz
[21144] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: Purusotha puram (0203041014), Year: 2017-2018 (2017-2018), Date: 10/01/2019 (1/10/2019 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041014_2017-2018_1_10_2019_12_00_00_AM_0.html.gz
[21145] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: Regupalem (0203041012), Year: 2023-2024 (2023-2024), Date: 02/09/2024 (9/2/2024 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041012_2023-2024_9_2_2024_12_00_00_AM_0.html.gz
[21146] Proces

[21168] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: Thurangalapalem (0203041003), Year: 2021-2022 (2021-2022), Date: 08/09/2022 (9/8/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041003_2021-2022_9_8_2022_12_00_00_AM_0.html.gz
[21169] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: Thurangalapalem (0203041003), Year: 2020-2021 (2020-2021), Date: 07/09/2022 (9/7/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041003_2020-2021_9_7_2022_12_00_00_AM_0.html.gz
[21170] Processed: State: ANDHRA PRADESH (02), District: ANAKAPALLI (0215), Block: Yelamanchili (0203041), Panchayat: Thurangalapalem (0203041003), Year: 2019-2020 (2019-2020), Date: 06/09/2022 (9/6/2022 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0215_0203041_0203041003_2019-2020_9_6_2022_12_00_00_AM_0.html.gz
Intermedia

[21187] Processed: State: ANDHRA PRADESH (02), District: ANANTAPUR (0212), Block: Anantapur (0212025), Panchayat: AKUTHOTAPALLI (0212025021), Year: 2020-2021 (2020-2021), Date: 15/12/2021 (12/15/2021 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0212_0212025_0212025021_2020-2021_12_15_2021_12_00_00_AM_0.html.gz
[21188] Processed: State: ANDHRA PRADESH (02), District: ANANTAPUR (0212), Block: Anantapur (0212025), Panchayat: AKUTHOTAPALLI (0212025021), Year: 2019-2020 (2019-2020), Date: 16/12/2021 (12/16/2021 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0212_0212025_0212025021_2019-2020_12_16_2021_12_00_00_AM_0.html.gz
[21189] Processed: State: ANDHRA PRADESH (02), District: ANANTAPUR (0212), Block: Anantapur (0212025), Panchayat: AKUTHOTAPALLI (0212025021), Year: 2018-2019 (2018-2019), Date: 26/01/2020 (1/26/2020 12:00:00 AM), Option: All (0). HTML saved to ../data/html/02_0212_0212025_0212025021_2018-2019_1_26_2020_12_00_00_AM_0.html.gz
[21190] Processed: